<a href="https://colab.research.google.com/github/ErikSeras/usos_r_python/blob/main/analisis_espacial/004_manejo_GEDI_L1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Introducción a los datos GEDI L1B en Python__

Elaborado por: __Erik Seras__ (_er.seras.perez@gmail.com_)

Fecha: 06-11-2020


---

> Este notebook se pudo elaborar gracias a los siguientes tutoriales:
- [Getting Started with NASA Global Ecosystems Dynamics Investigation (GEDI) Lidar Data
](https://www.youtube.com/watch?v=UlrCC1Xp-wk&t=1954s)
- [LP DAAC Data User Resources
GEDI-tutorials](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/browse)

---

<p align='justify'>Este tutorial demuestra cómo trabajar con el producto de datos de forma de onda geolocalizada  (<a href='https://doi.org/10.5067/GEDI/GEDI01_B.001'>GEDI01_B.001</a>).</p>

<p align='justify'>La misión de Investigación de la Dinámica de los Ecosistemas Globales (<a href='https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/gedi-overview/'>GEDI</a>) tiene como objetivo caracterizar la estructura y la dinámica de los ecosistemas para permitir una cuantificación y una comprensión radicalmente mejoradas del ciclo del carbono y la biodiversidad de la Tierra. El instrumento GEDI produce observaciones de alcance láser de alta resolución de la estructura tridimensional de la Tierra. GEDI está adjunto a la Estación Espacial Internacional y recopila datos a nivel mundial entre latitudes 51.6 $ ^ {o} $ N y 51.6 $ ^ {o} $ S con la resolución más alta y el muestreo más denso de cualquier instrumento de detección de luz y rango (lidar) en órbita hasta la fecha. Land Processes Distributed Active Archive Center (LP DAAC) distribuye los productos GEDI Nivel 1 y Nivel 2. Los productos L1B y L2 GEDI se archivan y distribuyen en el formato de archivo HDF-EOS5.</p>

---

__Ejemplo de caso de uso__:

<p align='justify'>Este tutorial se desarrolló utilizando un caso de uso de ejemplo para un proyecto que está completando el Servicio de Parques Nacionales. __El objetivo del proyecto es utilizar datos GEDI L1B para observar formas de onda GEDI sobre el Parque Nacional Redwood en el norte de California__.</p>

<p align='justify'>Este tutorial mostrará cómo usar Python para abrir archivos GEDI L1B, visualizar la órbita completa de puntos GEDI (tomas), subconjunto a una región de interés, visualizar formas de onda completas GEDI y exportar subconjuntos de capas de conjuntos de datos científicos (SDS) GEDI como GeoJSON archivos que se pueden cargar en programas de software GIS y / o de detección remota.</p>

---

__Datos utilizados en el ejemplo__:

- **GEDI L1B Geolocated Waveform Data Global Footprint Level - [GEDI01_B.001](https://doi.org/10.5067/GEDI/GEDI01_B.001)**
    - _El propósito del conjunto de datos L1B es proporcionar formas de onda geolocalizadas y conjuntos de datos de soporte para cada disparo de láser para los ocho rayos GEDI. Esto incluye formas de onda corregidas y suavizadas, parámetros de geolocalización y correcciones geofísicas.._  
     - **Science Dataset (SDS) layers:**  
        - /geolocation/latitude_bin0  
        - /geolocation/longitude_bin0  
        - /shot_number  
        - /stale_return_flag  
        - /geolocation/degrade  
        - /rx_sample_count  
        - /rx_sample_start_index  
        - /rxwaveform  
        - /geolocation/elevation_bin0  
        - /geolocation/elevation_lastbin  
        - /geolocation/digital_elevation_model  

---

__Tópicos cubiertos__:

1. **Comenzar** <br>
    1.1 Importar paquetes <br>
    1.2 Configurar el entorno de trabajo y recuperar archivos <br>
2. **Importar e interpretar datos** <br>
    2.1 Abra un archivo GEDI HDF5 y lea los metadatos del archivo <br>
    2.2 Leer metadatos SDS y subconjuntos por haz <br>
3. **Visualizar una órbita GEDI** <br>
    3.1 Subconjunto por capa y creación de un geodataframe <br>
    3.2 Visualizar un Geodataframe <br>
4. **Subconjunto y visualización de formas de onda** <br>
    4.1 Importar y extraer formas de onda <br>
    4.2 Visualizar formas de onda <br>
5. **Filtro de calidad** <br>
6. **Trayectos de perfil de trama** <br>
    6.1 Subconjuntos de vigas transversales <br>
    6.2 Trazar transectos de forma de onda <br>
7. **Visualización espacial** <br>
    7.1 Importación, subconjunto y filtro de calidad de todas las vigas <br>
    7.2 Subconjunto espacial <br>
    7.3 Visualizar todas las vigas: elevación <br>
8. **Exportar subconjuntos como archivos GeoJSON** <br>

---

__Datos de ejemplo__:

Este tutorial utiliza la observación GEDI L1B del 19 de junio de 2019 (órbita 02932). Utilice el enlace a continuación para descargar el archivo directamente desde LP DAAC Data Pool:
  - https://e4ftl01.cr.usgs.gov/GEDI/GEDI01_B.001/2019.06.19/GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5 (7.87 GB)

Se requiere una cuenta de [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) para descargar los datos utilizados en este tutorial. Puede crear una cuenta en el enlace proporcionado. Y también es requerido obtener autorización a su cuenta para poder usar datos GEDI.

Necesitará descargar el archivo anterior en el mismo directorio que este Jupyter Notebook para ejecutar correctamente el código siguiente.

---

__Código fuente utilizado para generar este tutorial__:

El repositorio que contiene todos los archivos requeridos se encuentra en: https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/browse
- [Cuaderno Jupyter](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/browse/GEDI_L1B_Tutorial.ipynb)
- [Parque Nacional Redwood GeoJSON](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/browse/RedwoodNP.geojson)
     - Contiene el límite administrativo del Parque Nacional Redwood, disponible en: [Límites administrativos de las unidades del sistema de parques nacionales 31/12/2017 - Conjunto de datos de parques nacionales de NPS del activo de datos geoespaciales nacionales (NGDA)](https://irma.nps.gov/DataStore/DownloadFile/594958)

<div class = "alerta alerta-bloque alerta-advertencia">
<b> NOTA: </b> Este tutorial fue desarrollado para archivos GEDI L1B HDF-EOS5 y solo debe usarse para ese producto. </div>

---
# 1 Comenzar <a id="getstarted"></a>

## 1.1 Instalar archivos y paquetes necesarios

In [ ]:
# Archivos para análisis espacial
! sudo add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
! sudo apt-get update
! sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev
! sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev

In [ ]:
! pip install geopandas
! pip install Shapely
! pip install geoviews
! pip install bokeh

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Point
import geoviews as gv
from geoviews import opts, tile_sources as gvts
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

Directorios para guardar los elementos a trabajar.

In [ ]:
# Directorio para guardar la data cruda
! mkdir input
# Directorio para guardar lo generado por el código
! mkdir output

Pasos necesarios para descargar directamente los datos de Nasa Earthdata.

## 1.2 Acceder directamente a los archivos de GEDI

Para poder descargar directamente datos GEDI L1B deberá iniciar sesión en el entorno de Google Colab con su cuenta de usuario de Nasa EarthData.

https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse/EarthdataLoginSetup.py#13

In [ ]:
from netrc import netrc
from subprocess import Popen
from getpass import getpass
import os

In [ ]:
urs = 'urs.earthdata.nasa.gov'
prompts = [
    'Ingrese el nombre de usuario de inicio de sesión de NASA Earthdata \n(o cree una cuenta en urs.earthdata.nasa.gov): ',
    'Ingrese la contraseña de inicio de sesión de NASA Earthdata'
]

https://stackoverflow.com/questions/14460189/python-try-except-not-working

In [ ]:
# Determine si el archivo netrc existe y, de ser así, si incluye credenciales de inicio de sesión de NASA Earthdata

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except Exception:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

Descargar la data directamente con `wget`.

https://lpdaac.usgs.gov/resources/e-learning/how-access-lp-daac-data-command-line/

In [ ]:
# Son casi 8 GB, asi que esto tomará un buen tiempo, todo dependerá de su internet
! wget https://e4ftl01.cr.usgs.gov/GEDI/GEDI01_B.001/2019.06.19/GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5 -P input

Como el archivo a trabajar pesa casi 8GB, puede guardar en su Google Drive para no perder la info descargada.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# Guardar a una carpeta de Google Drive (puede demorar unos minutos)
! cp /content/input/GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5 "/content/drive/My Drive/Colab Notebooks/python_r_gis/data"

cp: cannot stat '/content/input/GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5': No such file or directory


En caso ya se tenga el archivo en Google Drive y quiera practicar posteriormente con el notebook.

In [ ]:
! cp "/content/drive/My Drive/Colab Notebooks/python_r_gis/data/GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5" /content/input

In [ ]:
# Obtener los nombres de los archivos
gediFiles = [g for g in os.listdir('input') if g.startswith('GEDI01_B') and g.endswith('.h5')]  # List all GEDI L1B .h5 files in inDir
gediFiles

['GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5']

# 2 Importar e interpretar datos

## 2.1 Abrir un archivo GEDI HDF5 y lea los metadatos del archivo

Leer el archivo usando `h5py`.

In [ ]:
L1B = 'GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5'
L1B

'GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5'

El formato estándar para los nombres de archivo GEDI es el siguiente:

> __GEDI01_B__: Nombre corto del producto <br>
__2019170155833__: Fecha y hora juliana de adquisición (YYYYDDDHHMMSS) <br>
__O02932__: Número de órbita <br>
__T02267__: Número de pista <br>
__02__: Tipo de sistema de determinación de posicionamiento y señalamiento  (PPDS) (00 es predecible, 01 rápido, 02 y superior es final) <br>
__003__: Número de versión de GOC SDS (software) <br>
__01__: Versión de producción de gránulos <br>

Leer un archivo GEDI HDF5 usando el paquete `h5py`.

In [ ]:
gediL1B = h5py.File('input/'+ L1B, 'r')

Navegue por el archivo HDF5 a continuación.

In [ ]:
list(gediL1B.keys())

['BEAM0000',
 'BEAM0001',
 'BEAM0010',
 'BEAM0011',
 'BEAM0101',
 'BEAM0110',
 'BEAM1000',
 'BEAM1011',
 'METADATA']

El archivo GEDI HDF5 contiene grupos en los que se almacenan datos y metadatos.
Primero, el grupo `METADATA` contiene los metadatos a nivel de archivo.

In [ ]:
list(gediL1B['METADATA'])

['DatasetIdentification']

Contiene información útil como la fecha de creación, PGEVersion y VersionID. A continuación, imprima los atributos de metadatos a nivel de archivo.

In [ ]:
for g in gediL1B['METADATA']['DatasetIdentification'].attrs: print(g) 

PGEVersion
VersionID
abstract
characterSet
creationDate
credit
fileName
language
originatorOrganizationName
purpose
shortName
spatialRepresentationType
status
topicCategory
uuid


In [ ]:
print(gediL1B['METADATA']['DatasetIdentification'].attrs['purpose'])

The purpose of the L1B dataset is to provide geolocated waveforms and supporting datasets for each laser shot for all eight GEDI beams.  This includes corrected and smoothed waveforms, geolocation parameters, and geophysical corrections.


## 2.2 Leer metadatos SDS y subconjuntos por haz

El instrumento GEDI consta de 3 láseres que producen un total de 8 transectos de tierra de haz. Los ocho grupos restantes contienen datos para cada uno de los ocho transectos de haces GEDI. Para obtener información adicional, asegúrese de consultar: https://gedi.umd.edu/instrument/specifications/.

In [ ]:
beamNames = [g for g in gediL1B.keys() if g.startswith('BEAM')]
beamNames

['BEAM0000',
 'BEAM0001',
 'BEAM0010',
 'BEAM0011',
 'BEAM0101',
 'BEAM0110',
 'BEAM1000',
 'BEAM1011']

Una pieza útil de metadatos para recuperar de cada transecto de haz es si se trata de un haz de máxima potencia o de cobertura.

In [ ]:
for g in gediL1B['BEAM0000'].attrs: print(g)

description
wp-l1a-to-l1b_githash
wp-l1a-to-l1b_version


In [ ]:
for b in beamNames: 
    print(f"{b} is a {gediL1B[b].attrs['description']}")

BEAM0000 is a Coverage beam
BEAM0001 is a Coverage beam
BEAM0010 is a Coverage beam
BEAM0011 is a Coverage beam
BEAM0101 is a Full power beam
BEAM0110 is a Full power beam
BEAM1000 is a Full power beam
BEAM1011 is a Full power beam


A continuación, elija uno de los haces de máxima potencia que se utilizarán para recuperar formas de onda GEDI L1B en la Sección 3.

In [ ]:
beamNames = ['BEAM0110']

Identifique todos los objetos en el archivo GEDI HDF5 a continuación.

In [ ]:
gediL1B_objs = []
gediL1B.visit(gediL1B_objs.append)                                           # Recuperar lista de conjuntos de datos
gediSDS = [o for o in gediL1B_objs if isinstance(gediL1B[o], h5py.Dataset)]  # Busque SDS relevante dentro del archivo de datos
[i for i in gediSDS if beamNames[0] in i][0:10]                              # Imprima los primeros 10 conjuntos de datos para la viga seleccionada

['BEAM0110/all_samples_sum',
 'BEAM0110/ancillary/master_time_epoch',
 'BEAM0110/ancillary/mean_samples',
 'BEAM0110/ancillary/smoothing_width',
 'BEAM0110/beam',
 'BEAM0110/channel',
 'BEAM0110/geolocation/altitude_instrument',
 'BEAM0110/geolocation/altitude_instrument_error',
 'BEAM0110/geolocation/bounce_time_offset_bin0',
 'BEAM0110/geolocation/bounce_time_offset_bin0_error']

# 3 Visualizar una órbita GEDI
En la siguiente sección, importe las capas SDS de GEDI L1B en un GeoDataFrame de GeoPandas para la viga especificada anteriormente.
Utilice `latitude_bin0` y `longitude_bin0` para crear un punto `shapely` para cada ubicación de disparo GEDI.

## 3.1 Subconjunto por capa y crear un geodataframe

Leer la SDS y tome una muestra representativa (cada toma 100) y añádala a las listas, luego use las listas para generar un data frame de `pandas`.

In [ ]:
lonSample, latSample, shotSample, srfSample, degradeSample, beamSample = [], [], [], [], [], []  # Set up lists to store data

# Open the SDS
lats = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][()]
lons = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][()]
shots = gediL1B[f'{beamNames[0]}/shot_number'][()]
srf = gediL1B[f'{beamNames[0]}/stale_return_flag'][()]
degrade = gediL1B[f'{beamNames[0]}/geolocation/degrade'][()]

# Take every 100th shot and append to list
for i in range(len(shots)):
    if i % 100 == 0:
        shotSample.append(str(shots[i]))
        lonSample.append(lons[i])
        latSample.append(lats[i])
        srfSample.append(srf[i])
        degradeSample.append(degrade[i])
        beamSample.append(beamNames[0])
            
# Write all of the sample shots to a dataframe
latslons = pd.DataFrame({'Beam': beamSample, 'Shot Number': shotSample, 'Longitude': lonSample, 'Latitude': latSample,
                         'Stale Return Flag': srfSample, 'Degrade': degradeSample})
latslons

,Beam,Shot Number,Longitude,Latitude,Stale Return Flag,Degrade
0,BEAM0110,29320618800000001,111.996300,-51.803868,1,0
1,BEAM0110,29320604600000101,112.039132,-51.803905,1,0
2,BEAM0110,29320614600000201,112.080271,-51.803836,1,0
3,BEAM0110,29320600400000301,112.121445,-51.803737,1,0
4,BEAM0110,29320610400000401,112.162622,-51.803621,1,0
...,...,...,...,...,...,...
9792,BEAM0110,29320617400979201,88.208452,-51.803578,1,0
9793,BEAM0110,29320603200979301,88.249610,-51.803614,1,0
9794,BEAM0110,29320613200979401,88.290753,-51.803581,1,0
9795,BEAM0110,29320623200979501,88.331913,-51.803548,1,0


Arriba hay un marco de datos que contiene columnas que describen el haz, el número de disparos, la ubicación lat / lon e información de calidad sobre cada disparo.

In [ ]:
# Limpiar variables que ya no serán necesarias
del beamSample, degrade, degradeSample, gediL1B_objs, latSample, lats, lonSample, lons, shotSample, shots, srf, srfSample 

A continuación, cree una columna adicional llamada "geometría" que contenga un punto `shapely` generado a partir de cada ubicación lat / lon de la toma

In [ ]:
# Take the lat/lon dataframe and convert each lat/lon to a shapely point
latslons['geometry'] = latslons.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

A continuación, conviértalo en un GeoDataFrame de `Geopandas`.

In [ ]:
# Convert to a Geodataframe
latslons = gp.GeoDataFrame(latslons)
latslons = latslons.drop(columns=['Latitude','Longitude'])
latslons['geometry']

0       POINT (111.99630 -51.80387)
1       POINT (112.03913 -51.80391)
2       POINT (112.08027 -51.80384)
3       POINT (112.12145 -51.80374)
4       POINT (112.16262 -51.80362)
                   ...             
9792     POINT (88.20845 -51.80358)
9793     POINT (88.24961 -51.80361)
9794     POINT (88.29075 -51.80358)
9795     POINT (88.33191 -51.80355)
9796     POINT (88.37309 -51.80351)
Name: geometry, Length: 9797, dtype: geometry

Saque y trace un ejemplo de punto `shapely` a continuación.

In [ ]:
latslons['geometry'][0]

## 3.2 Visualizar un GeoDataFrame

En esta sección, utilice GeoDataFrame y el paquete python `geoviews` para visualizar espacialmente la ubicación de las tomas GEDI en un mapa base e importar un archivo geojson de la región espacial de interés para el ejemplo de caso de uso: Parque Nacional Redwood.

In [ ]:
# Define a function for visualizing GEDI points
def pointVisual(features, vdims):
    return (gvts.EsriImagery * gv.Points(features, vdims=vdims).options(
        tools=['hover'], height=500, width=900, size=5,
        color='yellow', fontsize={'xticks': 10, 'yticks': 10,'xlabel':16, 'ylabel': 16}
    ))

Importe un [geojson](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/browse/RedwoodNP.geojson) de Redwood National Park como un GeoDataFrame adicional.

In [ ]:
# Descargar el archivo .geojson
! wget -O RedwoodNP.geojson https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/raw/RedwoodNP.geojson?at=refs%2Fheads%2Fmaster

In [ ]:
# Importar geojson como GeoDataFrame
redwoodNP = gp.GeoDataFrame.from_file('RedwoodNP.geojson')

In [ ]:
redwoodNP

,GIS_LOC_ID,UNIT_CODE,GROUP_CODE,UNIT_NAME,UNIT_TYPE,META_MIDF,LANDS_CODE,DATE_EDIT,GIS_NOTES,geometry
0,None,REDW,None,Redwood,National Park,None,None,None,Shifted 0.06 miles,"MULTIPOLYGON (((-124.01829 41.44539, -124.0184..."


In [ ]:
redwoodNP['geometry'][0]

Definir los vdims a continuación que permitirá desplazarse sobre tomas específicas y ver información sobre ellas.

In [ ]:
# Create a list of geodataframe columns to be included as attributes in the output map
vdims = []
for f in latslons:
    if f not in ['geometry']:
        vdims.append(f)
vdims

['Beam', 'Shot Number', 'Stale Return Flag', 'Degrade']

A continuación, combine una parcela del límite del Parque Nacional Redwood (combine dos parcelas de `geoviews` usando `*`) con la función de mapeo visual de puntos definida anteriormente para trazar: <br>
(1) las tomas GEDI representativas, <br>
(2) la región de interés y <br>
(3) una capa de mapa base. <br>

In [ ]:
import holoviews as hv
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

# Call the function for plotting the GEDI points
gv.Polygons(redwoodNP).opts(line_color='red', color=None) * pointVisual(latslons, vdims = vdims)

Arriba hay una buena ilustración de la órbita GEDI completa (los archivos GEDI se almacenan como una órbita ISS). Uno de los beneficios de utilizar las vistas geográficas es la naturaleza interactiva de los gráficos de salida. Use las herramientas a la derecha del mapa de arriba para acercar y encontrar las tomas que se cruzan con el Parque Nacional Redwood.
> (SUGERENCIA: encuentre dónde la órbita se cruza con la costa oeste de los Estados Unidos)

A continuación se muestra una captura de pantalla de la región de interés:
![alt text](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/raw/GEDI_L1B_Tutorial_1.png?at=refs%2Fheads%2Fmaster "Sample of GEDI L1B shots in yellow (orbit 2932) plotted over Redwood National Park, USA.")

> Nota al margen: ¿Se pregunta qué significan los 0 y los 1 de stale_return_flag y degrade?

In [ ]:
print(f"stale_return_flag: {gediL1B[b]['stale_return_flag'].attrs['description']}")
print(f"degrade: {gediL1B[b]['geolocation']['degrade'].attrs['description']}")

stale_return_flag: Indicates that a "stale" cue point from the coarse search algorithm is being used.
degrade: Greater than zero if the shot occurs during a degrade period, zero otherwise.


Mostraremos un ejemplo de cómo filtrar la calidad de los datos GEDI en la sección 5.

Después de encontrar uno de los disparos dentro de Redwood NP, busque el índice de ese número de disparos para que podamos encontrar la forma de onda correcta para visualizar en la Sección 4.

Cada disparo GEDI tiene un identificador de disparo único (número de disparo) que está disponible dentro de cada grupo de datos del producto. Es importante conservar el número de disparo en cualquier subconjunto de datos, ya que permitirá al usuario vincular cualquier registro de disparo con los datos de la órbita original y vincular cualquier disparo y sus datos entre los productos L1 y L2. El formato estándar para los disparos GEDI es el siguiente:

Disparo: 29320619900465601
> 2932: número de órbita <br>
06: Número de haz <br>
199: Número de fotograma menor (0-241) <br>
00465601: número de disparo dentro de la órbita <br>

In [ ]:
shot = 29320619900465601

In [ ]:
index = np.where(gediL1B[f'{beamNames[0]}/shot_number'][()]==shot)[0][0]  # Set the index for the shot identified above
index

465600

In [ ]:
del latslons  # No longer need the geodataframe used to visualize the full GEDI orbit

# 4 Crear subconjuntos y visualizar formas de onda

En esta sección, aprenderá a extraer y crear subconjuntos de formas de onda específicas y trazarlas mediante `holoviews`.

## 4.1 Importar y extraer formas de onda

Para encontrar y extraer la forma de onda completa para el índice exacto que nos interesa, en lugar de importar el conjunto de datos de forma de onda completo (¡más de mil millones de valores!), Usaremos `rx_sample_count` y `rx_sample_start_index` para identificar la ubicación de la forma de onda en la que estamos interesado en visualizar y simplemente extraer esos valores.

In [ ]:
# From the SDS list, use list comprehension to find sample_count, sample_start_index, and rxwaveform
sdsCount = gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_count') and beamNames[0] in g][0]]
sdsStart = gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_start_index') and beamNames[0] in g][0]]
sdsWaveform = [g for g in gediSDS if g.endswith('/rxwaveform') and beamNames[0] in g][0]

Imprima la descripción de cada uno de estos conjuntos de datos para comprender mejor cómo los usaremos para extraer formas de onda específicas.

In [ ]:
print(f"rxwaveform is {gediL1B[sdsWaveform].attrs['description']}")

rxwaveform is The corrected receive (RX) waveforms.  Use rx_sample_count and rx_sample_start_index to identify the location of each waveform.


A continuación, lea cómo usar las capas `rx_sample_count` y `rx_sample_start_index`:

In [ ]:
print(f"rx_sample_count is {sdsCount.attrs['description']}")
print(f"rx_sample_start_index is {sdsStart.attrs['description']}")

rx_sample_count is The number of sample intervals (elements) in each RX waveform.
rx_sample_start_index is The index in the rxwaveform dataset of the first element of each RX waveform.  The indices start at 1.


Utilice `rx_sample_count` y `rx_sample_start_index` para identificar la ubicación de cada forma de onda en `rxwaveform`.

In [ ]:
wfCount = sdsCount[index]           # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)  # Subtract one because python array indexing starts at 0 not 1

A continuación, obtenga información adicional sobre la toma, incluido el único `shot_number` y la ubicación lat / lon.

In [ ]:
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]

Ponga todo junto para identificar la forma de onda que queremos extraer:

In [ ]:
print(f"The waveform located at: {str(wfLat)}, {str(wfLon)} (shot ID: {wfShot}, index {index}) is from beam {beamNames[0]} \
      and is stored in rxwaveform beginning at index {wfStart} and ending at index {wfStart + wfCount}")

The waveform located at: 41.28533109208681, -124.0300090357876 (shot ID: 29320619900465601, index 465600) is from beam BEAM0110       and is stored in rxwaveform beginning at index 661152000 and ending at index 661152812


Para trazar una forma de onda completa, también necesita importar la elevación registrada en `bin0` (la primera elevación registrada) y `lastbin` o la última elevación registrada para esa forma de onda.

In [ ]:
# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

Extraiga la forma de onda completa usando el índice de inicio y recuento:

A continuación, puede ver por qué es importante extraer la forma de onda específica que le interesa: ¡casi 1.4 mil millones de valores están almacenados en el conjunto de datos rxwaveform!

In [ ]:
print("{:,}".format(gediL1B[sdsWaveform].shape[0]))

1,391,172,580


In [ ]:
# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

## 4.2 Visualizar formas de onda

A continuación, traza la forma de onda extraída usando la diferencia de elevación desde bin0 hasta last_bin en el eje y, y la energía de forma de onda devuelta o amplitud (número digital = dn) en el eje x.

In [ ]:
# Find elevation difference from start to finish and divide into equal intervals based on sample_count
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

In [ ]:
# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})
wvDF

,Amplitude (DN),Elevation (m)
0,226.009323,37.228645
1,225.360245,37.078965
2,225.373764,36.929286
3,226.165710,36.779607
4,227.480103,36.629927
...,...,...
807,231.473923,-83.562517
808,230.386932,-83.712196
809,228.637512,-83.861876
810,226.949661,-84.011555


A continuación, use el paquete de Python `Holoviews` (hv) para trazar la forma de onda.

https://stackoverflow.com/questions/48295229/bokeh-inside-google-colab

In [ ]:
import holoviews as hv
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

hv.Curve(wvDF) # Basic line graph plotting the waveform

¡Felicidades! Ha trazado su primera forma de onda.

Arriba hay una gráfica de línea básica que muestra la amplitud (DN) en función de la elevación de la forma de onda rx para el disparo específico seleccionado. A continuación, agregue elementos de gráfico adicionales y haga que el gráfico sea interactivo.

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()


# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
wfVis = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
wfVis

Como puede ver, la toma seleccionada no parece tan interesante: esta forma de onda parece más característica de algún tipo de copa baja / suelo desnudo que la copa de un árbol. Si hace zoom en la toma GEDI, parece estar sobre el río o posiblemente una barra de arena:

![texto alternativo](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/raw/GEDI_L1B_Tutorial_2.png?at=refs%2Fheads%2Fmaster "GEDI L1B Disparo sobre un río en el Parque Nacional Redwood, EE. UU.")
Luego, traza un par de formas de onda más y mira si puedes capturar una toma sobre el bosque.

In [ ]:
latlons = {}                          # Set up a dictionary to hold multiple waveforms
latlons[wfShot] = Point(wfLon,wfLat)  # Create shapely point and add to dictionary

# Retain waveform and quality layers to be exported later
latlonsWF = [waveform]
latlonsEL = [zStretch]
latlonsSRF = [gediL1B[f'{beamNames[0]}/stale_return_flag'][index]]
latlonsD = [gediL1B[f'{beamNames[0]}/geolocation/degrade'][index]]

In [ ]:
# Define which observation to examine by selecting the three indexes prior to the one used above
index = 465597

In [ ]:
# Use rx_sample_count and rx_sample_start_index to identify the location of each waveform
wfCount = sdsCount[index]                                             # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)                                    # Subtract 1, index starts at 0 not 1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]                # Unique Shot Number
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]   # Latitude
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]  # Longitude
latlons[wfShot] = Point(wfLon,wfLat)                                  # Create shapely point and add to dictionary

# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

In [ ]:
# Find elevation difference from start to finish, divide into equal intervals
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})

In [ ]:
# Append waveform and quality layers to be exported later
latlonsWF.append(waveform)
latlonsEL.append(zStretch)
latlonsSRF.append(gediL1B[f'{beamNames[0]}/stale_return_flag'][index])
latlonsD.append(gediL1B[f'{beamNames[0]}/geolocation/degrade'][index])

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
visL1B1 = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
visL1B1

¡Ahora eso está comenzando a parecerse más a un dosel de árbol muy alto y de varias capas! Continúe con la siguiente toma:

In [ ]:
# Define which observation to examine by selecting the three indexes prior to the one used above
index = 465598

In [ ]:
# Use rx_sample_count and rx_sample_start_index to identify the location of each waveform
wfCount = sdsCount[index]                                             # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)                                    # Subtract 1, index starts at 0 not 1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]                # Unique Shot Number
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]   # Latitude
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]  # Longitude
latlons[wfShot] = Point(wfLon,wfLat)                                  # Create shapely point and add to dictionary

# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

In [ ]:
# Find elevation difference from start to finish, divide into equal intervals
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})

In [ ]:
# Append waveform and quality layers to be exported later
latlonsWF.append(waveform)
latlonsEL.append(zStretch)
latlonsSRF.append(gediL1B[f'{beamNames[0]}/stale_return_flag'][index])
latlonsD.append(gediL1B[f'{beamNames[0]}/geolocation/degrade'][index])

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
visL1B2 = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
visL1B2

In [ ]:
# Define which observation to examine by selecting the three indexes prior to the one used above
index = 465599

In [ ]:
# Use rx_sample_count and rx_sample_start_index to identify the location of each waveform
wfCount = sdsCount[index]                                             # Number of samples in the waveform
wfStart = int(sdsStart[index] - 1)                                    # Subtract 1, index starts at 0 not 1
wfShot = gediL1B[f'{beamNames[0]}/shot_number'][index]                # Unique Shot Number
wfLat = gediL1B[f'{beamNames[0]}/geolocation/latitude_bin0'][index]   # Latitude
wfLon = gediL1B[f'{beamNames[0]}/geolocation/longitude_bin0'][index]  # Longitude
latlons[wfShot] = Point(wfLon,wfLat)                                  # Create shapely point and add to dictionary

# Grab the elevation recorded at the start and end of the full waveform capture
zStart = gediL1B[f'{beamNames[0]}/geolocation/elevation_bin0'][index]   # Height of the start of the rx window
zEnd = gediL1B[f'{beamNames[0]}/geolocation/elevation_lastbin'][index]  # Height of the end of the rx window

# Retrieve the waveform sds layer using the sample start index and sample count information to slice the correct dimensions
waveform = gediL1B[sdsWaveform][wfStart: wfStart + wfCount]

In [ ]:
# Find elevation difference from start to finish, divide into equal intervals
zStretch = np.add(zEnd, np.multiply(range(wfCount, 0, -1), ((zStart - zEnd) / int(wfCount))))

# match the waveform amplitude values with the elevation and convert to Pandas df
wvDF = pd.DataFrame({'Amplitude (DN)': waveform, 'Elevation (m)': zStretch})

In [ ]:
# Append waveform and quality layers to be exported later
latlonsWF.append(waveform)
latlonsEL.append(zStretch)
latlonsSRF.append(gediL1B[f'{beamNames[0]}/stale_return_flag'][index])
latlonsD.append(gediL1B[f'{beamNames[0]}/geolocation/degrade'][index])

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

# Create a holoviews interactive Curve plot with additional parameters defining the plot aesthetics 
visL1B3 = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=500, width=400,
           xlim=(np.min(waveform) - 10, np.max(waveform) + 10), ylim=(np.min(zStretch), np.max(zStretch)),
           fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(wfShot)}')
visL1B3

Por último, trace el transecto de cuatro formas de onda consecutivas:

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

# The "+" symbol will plot multiple saved holoviews plots together
visL1B1.opts(width=240) + visL1B2.opts(width=240, labelled=[]) + visL1B3.opts(width=240, labelled=[]) + \
wfVis.opts(width=240, labelled=[])

Observe que se mueve de oeste a este (de izquierda a derecha) a lo largo del transecto; puede ver que la elevación desciende y el dosel de los árboles disminuye a medida que se encuentra con el banco de arena / río.

A continuación, use las vistas geográficas para trazar la ubicación de los cuatro puntos y verificar lo que se ve arriba.

In [ ]:
# Convert dict to geodataframe
latlons_gp = gp.GeoDataFrame({
    'Shot Number': list(latlons.keys()),'rxwaveform Amplitude (DN)': latlonsWF,
    'rxwaveform Elevation (m)': latlonsEL, 'Stale Return Flag': latlonsSRF, 'Degrade': latlonsD,
    'geometry': list(latlons.values())
})

In [ ]:
latlons_gp['Shot Number'] = latlons_gp['Shot Number'].astype(str)  # Convert shot number from integer to string

In [ ]:
vdims

['Beam', 'Shot Number', 'Stale Return Flag', 'Degrade']

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

# Create a list of geodataframe columns to be included as attributes in the output map
vdims = []
for f in latlons_gp:
    if f not in ['geometry']:
        if 'rxwaveform' not in f:
            vdims.append(f)

# Plot the geodataframe
gv.Polygons(redwoodNP).opts(line_color='#6de3e1', color=None) * pointVisual(latlons_gp, vdims=vdims)

Arriba, amplíe los puntos amarillos hasta que pueda obtener una vista más clara de las cuatro tomas.

En las capturas de pantalla a continuación, podemos hacer coincidir la ubicación de cada punto con la forma de onda asociada. Parece confirmar la hipótesis de que la elevación está disminuyendo a medida que los disparos se acercan al río, y el dosel del bosque disminuye hasta que detecta la barra de arena / río por la forma de onda final en la secuencia.

![alt text](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/raw/GEDI_L1B_Tutorial_3.png?at=refs%2Fheads%2Fmaster "Plots of Four GEDI L1B Waveforms")
![alt text](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/raw/GEDI_L1B_Tutorial_4.png?at=refs%2Fheads%2Fmaster "Map showing sequence of four GEDI L1B Waveforms in Redwood National Park, USA.")

In [ ]:
del wfVis, visL1B1, visL1B2, visL1B3, waveform, wvDF, zStretch, latlonsWF, latlonsEL, latlonsSRF, latlonsD

# 5 Filtrado de calidad

Ahora que tiene las capas deseadas importadas como un marco de datos para las tomas seleccionadas, realicemos un filtrado de calidad.

A continuación, elimine todas las tomas en las que `stale_return_flag` esté establecido en 1 (indica que se está utilizando un punto de referencia "obsoleto" del algoritmo de búsqueda aproximada) definiendo esas tomas como `nan`.

La sintaxis de la línea a continuación se puede leer como: en el marco de datos, busque las filas "donde" el indicador de retorno obsoleto no es igual (ne) a 0. Si una fila (toma) no cumple la condición, establezca todos los valores iguales a `nan` para esa fila.

In [ ]:
latlons_gp = latlons_gp.where(latlons_gp['Stale Return Flag'].ne(1))  # Set any stale returns to NaN

A continuación, filtre la calidad aún más usando la bandera de `degrade` (mayor que cero si el disparo ocurre durante un período de degradación, cero en caso contrario).

In [ ]:
latlons_gp = latlons_gp.where(latlons_gp['Degrade'].ne(1))

A continuación, elimine todas las tomas que no pasaron los estándares de filtrado de calidad descritos anteriormente del `transectDF`.

In [ ]:
latlons_gp = latlons_gp.dropna()  # Drop all of the rows (shots) that did not pass the quality filtering above

In [ ]:
print(f"Quality filtering complete, {len(latlons_gp)} high quality shots remaining.")

Quality filtering complete, 4 high quality shots remaining.


¡Buenas noticias! Parece que las cuatro formas de onda de ejemplo pasaron las pruebas iniciales de filtrado de calidad. Para obtener información adicional sobre el filtrado de calidad de los datos GEDI, asegúrese de consultar: https://lpdaac.usgs.gov/resources/faqs/#how-should-i-quality-filter-gedi-l1b-l2b-data.

In [ ]:
del latlons_gp

# 6 Trayectos de perfil de parcela

En esta sección, trace un subconjunto de transectos utilizando formas de onda.

## 6.1 Subconjuntos de haces de haz

Subconjunto hasta un transecto más pequeño centrado en las formas de onda analizadas en las secciones anteriores.

In [ ]:
print(index)

465599


In [ ]:
# Grab 50 points before and after the shot visualized above
start = index - 50
end = index + 50 
transectIndex = np.arange(start, end, 1)

# 6.2 Trazar transectos de forma de onda

Para tener una idea de la longitud del transecto de la viga que está trazando, puede trazar el eje x como distancia, que se calcula a continuación.

In [ ]:
# Calculate along-track distance
distance = np.arange(0.0, len(transectIndex) * 60, 60)  # GEDI Shots are spaced 60 m apart

Para trazar cada valor vertical para cada forma de onda, deberá reformatear la estructura de datos para que coincida con lo que necesitan las capacidades de `holoviews` Path ().

In [ ]:
# Create a list of tuples containing Shot number, and each waveform value at height (z)
wfList = []
for s, i in enumerate(transectIndex):
    # Basic quality filtering from section 5
    if gediL1B['BEAM0110/geolocation/degrade'][i] == 0 and gediL1B['BEAM0110/stale_return_flag'][i] == 0:
        zStart = gediL1B['BEAM0110/geolocation/elevation_bin0'][i]
        zEnd = gediL1B['BEAM0110/geolocation/elevation_lastbin'][i]
        zCount = sdsCount[i]
        zStretch = np.add(zEnd, np.multiply(range(zCount, 0, -1), ((zStart - zEnd) / int(zCount))))
        waveform = gediL1B[sdsWaveform][sdsStart[i]: sdsStart[i] + zCount]
        waves = []
        for z, w in enumerate(waveform):
            waves.append((distance[s], zStretch[z],w))  # Append Distance (x), waveform elevation (y) and waveform amplitude (z)
        wfList.append(waves)
    else:
        print(f"Shot {s} did not pass quality filtering and will be excluded.")

Buenas noticias de nuevo, parece que todas las formas de onda de nuestro transecto pasaron la prueba de filtrado de calidad.
A continuación, trace cada forma de onda utilizando la función Path () de `holoviews`. Esto trazará cada valor de forma de onda individual por distancia, con la amplitud trazada en la tercera dimensión en tonos de verde.

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

# Plot the waveform values for the transect with the waveform amplitude being plotted as the 'c' or colormap
l1bVis = hv.Path(wfList, vdims='Amplitude').options(
    color='Amplitude', clim=(200,405), cmap='Greens', line_width=20, width=950,
    height=500, clabel='Amplitude',colorbar=True,
    xlabel='Distance Along Transect (m)', ylabel='Elevation (m)',
    fontsize={
        'title':16, 'xlabel':16, 'ylabel': 16, 'xticks':12, 'yticks':12,
        'clabel':12, 'cticks':10
    },
    colorbar_position='right',
    title='GEDI L1B Waveforms across Redwood National Park: June 19, 2019'
)
l1bVis

Si hace zoom en un subconjunto del transecto, puede obtener aún mejores detalles que muestran la estructura del dosel del conjunto de datos `rxwaveform`, donde las porciones más densas del dosel se ven en tonos más oscuros de verde.

![alt text](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-tutorials/raw/GEDI_L1B_Tutorial_5.png?at=refs%2Fheads%2Fmaster "Full waveform amplitude plotted for a transect covering Redwood National Park, USA.")

In [ ]:
del waveform, waves, wfList, zStretch

# 7 Visión espacial

La Sección 7 combina muchas de las técnicas aprendidas anteriormente, incluyendo cómo importar conjuntos de datos GEDI, realizar filtrado de calidad, subconjuntos espaciales y visualización.

## 7.1 Importación, subconjunto y filtro de calidad de todas las haz

A continuación, vuelva a abrir la observación GEDI L1B, pero esta vez, recorra e importe los datos de los 8 haz GEDI.

In [ ]:
beamNames = [g for g in gediL1B.keys() if g.startswith('BEAM')]

In [ ]:
beamNames

['BEAM0000',
 'BEAM0001',
 'BEAM0010',
 'BEAM0011',
 'BEAM0101',
 'BEAM0110',
 'BEAM1000',
 'BEAM1011']

Recorra cada uno de los conjuntos de datos deseados (SDS) para cada haz, añádalos a las listas y transfórmese en un DataFrame de `pandas`.

In [ ]:
# Set up lists to store data
shotNum, dem, zElevation, zHigh, zLat, zLon, srf, degrade, beamI, rxCount, rxStart = ([] for i in range(11))

In [ ]:
# Loop through each beam and open the SDS needed
for b in beamNames:
    [shotNum.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()]]
    [dem.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/digital_elevation_model') and b in g][0]][()]]
    [zElevation.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/elevation_bin0') and b in g][0]][()]]  
    [zHigh.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/elevation_lastbin') and b in g][0]][()]]  
    [zLat.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/latitude_bin0') and b in g][0]][()]]  
    [zLon.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/longitude_bin0') and b in g][0]][()]]  
    [srf.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/stale_return_flag') and b in g][0]][()]]  
    [degrade.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/degrade') and b in g][0]][()]]  
    [rxCount.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_count') and b in g][0]][()]]  
    [rxStart.append(h) for h in gediL1B[[g for g in gediSDS if g.endswith('/rx_sample_start_index') and b in g][0]][()]]  
    [beamI.append(h) for h in [b] * len(gediL1B[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()])]  

Tenga en cuenta que no importamos el conjunto de datos `rxwaveform`; debido al gran tamaño de ese conjunto de datos, esperaremos hasta después del subconjunto espacial para importar las formas de onda.

In [ ]:
# Convert lists to Pandas dataframe
allDF = pd.DataFrame({
    'Shot Number': shotNum, 'Beam': beamI, 'Latitude': zLat, 'Longitude': zLon,
    'Tandem-X DEM': dem, 'Elevation bin 0 (m)': zElevation,
    'Elevation last bin (m)': zHigh, 'Stale Return Flag': srf,
    'Degrade Flag': degrade, 'rx Start Index': rxStart, 'rx Sample Count': rxCount
})

In [ ]:
del shotNum, dem, zElevation, zHigh, zLat, zLon, srf, degrade, beamI

## 7.2 Subconjunto espacial

A continuación, realice un subconjunto del marco de datos de pandas utilizando una región de interés del cuadro delimitador simple. Si está interesado en recortar espacialmente tomas GEDI a una región geojson de interés, asegúrese de consultar el script de Python GEDI-Subsetter disponible en: https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse

In [ ]:
len(allDF)

3547051

¡Más de 3.5 millones de disparos están contenidos en esta única órbita GEDI! A continuación, subconjunto hasta solo las tomas que caen dentro de este pequeño cuadro delimitador que abarca el Parque Nacional Redwood. RedwoodNP nuestro geodataframe de geopandas se puede llamar para el "sobre" o cuadro delimitador más pequeño que abarca toda la región de interés. Aquí, utilícelo como cuadro delimitador para subconjuntos de las tomas GEDI.

In [ ]:
redwoodNP.envelope[0].bounds

(-124.16015705494489,
 41.080601363502545,
 -123.84950230520286,
 41.83981133687605)

In [ ]:
minLon, minLat, maxLon, maxLat = redwoodNP.envelope[0].bounds  # Define the min/max lat/lon from the bounds of Redwood NP

Filtre por el cuadro delimitador, que se hace de manera similar al filtrado por calidad en la sección 5 anterior.

In [ ]:
allDF = allDF.where(allDF['Latitude'] > minLat)
allDF = allDF.where(allDF['Latitude'] < maxLat)
allDF = allDF.where(allDF['Longitude'] > minLon)
allDF = allDF.where(allDF['Longitude'] < maxLon)

In [ ]:
allDF = allDF.dropna()  # Drop shots outside of the ROI

In [ ]:
len(allDF)

4477

Observe que ha reducido drásticamente la cantidad de tomas con las que está trabajando (lo que mejorará en gran medida su experiencia al trazarlas a continuación). Pero primero, elimine las tomas de mala calidad que existan dentro del ROI.

In [ ]:
# Set any poor quality returns to NaN
allDF = allDF.where(allDF['Stale Return Flag'].ne(1))
allDF = allDF.where(allDF['Degrade Flag'].ne(1))
allDF = allDF.dropna()
len(allDF)

4475

A continuación, cree un punto `Shapely` de cada plano e insértelo como columna de geometría en data frame (que luego será GeoDataFrame).

In [ ]:
# Take the lat/lon dataframe and convert each lat/lon to a shapely point
allDF['geometry'] = allDF.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

In [ ]:
# Convert to geodataframe
allDF = gp.GeoDataFrame(allDF)
allDF = allDF.drop(columns=['Latitude','Longitude'])

## 7.3 Visualizar todas los haz: elevación

Ahora, usando la función `pointVisual` definida en la sección 3.2, trace el GeoDataFrame de `geopandas` usando `geoviews`.

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

allDF['Shot Number'] = allDF['Shot Number'].astype(str)  # Convert shot number to string

vdims = []
for f in allDF:
    if f not in ['geometry']:
        vdims.append(f)

visual = pointVisual(allDF, vdims = vdims)
visual * gv.Polygons(redwoodNP).opts(line_color='red', color=None)

Siéntase libre de desplazarse y acercarse a las tomas GEDI en amarillo.

Ahora no solo tracemos los puntos en el GeoDataFrame, sino que también agreguemos un mapa de colores para Elevation bin0 (m).

In [ ]:
# Necesario para mostrarlo en Google Colab
hv.extension('bokeh')
output_notebook()

(
    gvts.EsriImagery * gv.Points(allDF, vdims=vdims).options(
        color='Elevation bin 0 (m)',cmap='terrain', size=3, tools=['hover'],
        clim=(min(allDF['Elevation bin 0 (m)']), max(allDF['Elevation bin 0 (m)'])),
        colorbar=True, clabel='Meters', title='GEDI Elevation bin0 over Redwood National Park: June 19, 2019',
        fontsize={'xticks': 10, 'yticks': 10, 'xlabel':16, 'clabel':12,'cticks':10,'title':16,'ylabel':16}
    )
).options(height=500,width=900)

# 8 Exportar subconjuntos como archivos GeoJSON

En esta sección, exporte el GeoDataFrame como un archivo .geojson que se puede abrir fácilmente en su software favorito de teledetección y / o GIS e incluirá una tabla de atributos con todas las tomas / valores para cada una de las capas SDS en el marco de datos.

In [ ]:
waves = []  # List to store each list of waveform values

# Loop through each row (shot) in the data frame, extract waveform values, append to list of lists
for _, shot in allDF.iterrows():
    starts = int(shot['rx Start Index'])
    wave = gediL1B[f"{shot['Beam']}/rxwaveform"][starts: starts + int(shot['rx Sample Count'])]
    
    # In order to export as geojson, the type needs to be converted from a numpy array to a single comma separated string
    waves.append(','.join([str(q) for q in wave]))
    
# Add to data frame
allDF['rx Waveform'] = waves

In [ ]:
gediL1B.filename  # L1B Filename

'input/GEDI01_B_2019170155833_O02932_T02267_02_003_01.h5'

In [ ]:
outName = gediL1B.filename.replace('.h5', '.json')  # Create an output file name using the input file name
outName

'input/GEDI01_B_2019170155833_O02932_T02267_02_003_01.json'

In [ ]:
allDF.to_file(outName, driver='GeoJSON')  # Export to GeoJSON

In [ ]:
del allDF, waves

¡Éxito! Ahora ha aprendido cómo comenzar a trabajar con archivos GEDI L1B en Python (en el entorno de Google Colab), así como algunas estrategias interesantes para visualizar esos datos a fin de comprender mejor su región específica de interés. Al usar este cuaderno jupyter como flujo de trabajo, ahora debería poder cambiar a archivos GEDI en su región específica de interés y volver a ejecutar el cuaderno. ¡Buena suerte!